In [ ]:
import numpy as np
import pandas as pd
import dcarte
import os
import math
import seaborn as sns
import warnings
from numpy import log2
from argparse import Namespace
import torch
import torch.nn as nn
from scipy import stats
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from misc.sampler import CartesianSampler
from toy.ratchet import simulation, ep_per_step, analytic_etpy
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings('ignore')
dcarte.domains()

In [ ]:
time = 0  # daytime:0 (6:00-18:00), night:1 (18:00-6:00)
timestep = 0  # per day: 0, per week: 1, per hour: 2, accurate time: 3
activity_raw = dcarte.load('Activity','RAW')
activity_legacy = dcarte.load('Motion','LEGACY')

In [ ]:
def pre_process_raw(activity_raw):
    
    # delete
    activity = activity_raw
    
    # revise location names
    activity = activity.copy()
    mapping = {
    'conservatory':'conservatory',
    'WC1':'wc',
    'corridor1':'corridor',
    'living room':'living',
    'study':'study',
    'dining room':'dining',
    'bathroom1':'bathroom',
    'bedroom1':'bedroom',
    'hallway':'hallway',
    'lounge':'lounge',
    'kitchen':'kitchen',
    'cellar':'cellar',
    'office':'office'
    }
    activity.location_name = activity.location_name.map(mapping)
    activity = activity[~activity['location_name'].isin(['cellar','office','dining','study','living','corridor','wc','conservatory'])]
    activity.location_name = activity.location_name.values.astype('str')
    activity.patient_id = activity.patient_id.values.astype('str')
    
    # delete rebundant columns
    activity.drop(['home_id','location_id','source'],axis=1, inplace=True)
    
    return activity

In [ ]:
def pre_process_legacy(activity_raw):
    
    # delete
    activity = activity_raw
    
    # revise location names
    activity = activity.copy()
    mapping = {
    'Hallway':'hallway',
    'Kitchen':'kitchen',
    'Study':'study',
    'Bathroom':'bathroom',
    'Lounge':'lounge',
    'Bedroom':'bedroom',
    'Living Room':'living',
    'Front Door':'door',
    'D':'d',
    'Dining Room':'dining',
    }
    activity.location_name = activity.location_name.map(mapping)
    activity = activity[~activity['location_name'].isin(['study','living','door','d','dining'])]
    activity.location_name = activity.location_name.values.astype('str')
    activity.patient_id = activity.patient_id.values.astype('str')
    
    # delete rebundant columns
    activity.drop(['index','timezone'],axis=1, inplace=True)
    activity = activity[['start_date','patient_id','location_name']]
    
    return activity

In [ ]:
def select_daytime_night(my_activity,my_time):
    
    # daytime:0 (6:00-18:00), night:1 (18:00-6:00)
    signal = my_time
    
    if signal==0:
        print("Time: daytime")
        activity_day = my_activity
        activity_day['hour'] = activity_day.start_date.dt.hour
        # choose daytime, between [6:00-18:00]
        activity_day = activity_day[activity_day['hour'].between(6,17)]
        activity_day = activity_day.copy()
        activity_day.drop(['hour'],axis=1, inplace=True)
        activity_day['day_date'] =  activity_day.start_date.values.astype("datetime64[D]")
        activity_select = activity_day
        
    elif signal==1:
        print("Time: night")
        activity_night = my_activity
        activity_night['hour'] = activity_night.start_date.dt.hour
        # choose night time, except [6:00-18:00]. e.g., the night time on 22/3 includes 18:00-24:00 on 22/3 and 00:00-06:00 on 23/3
        activity_night = activity_night[~activity_night['hour'].between(6,17)]
        activity_night = activity_night.copy()
        activity_night['day_date'] = activity_night.start_date.values.astype("datetime64[D]")
        activity_night['last_date'] = activity_night['start_date'] + pd.Timedelta(days=-1)
        activity_night['day_date'] =  activity_night['day_date'].mask(activity_night['hour']<6, activity_night['last_date'])
        activity_night['day_date'] = activity_night.day_date.values.astype("datetime64[D]")
        activity_night.drop(['hour','last_date'],axis=1, inplace=True)
        activity_select = activity_night
        
    else:
        raise ValueError("Error: please input correct number! daytime:0 (6:00-18:00), night:1 (18:00-6:00)")
        
    return activity_select

In [ ]:
def select_time_step(my_activity, my_timestep):
    
    activity = pd.DataFrame(my_activity)
    # per day: 0, per week: 1, per hour: 2, accurate time: 3
    signal = my_timestep
    
    if signal==0:
        print("Timestep: per day")
        activity.day_date = activity.day_date.values.astype("datetime64[D]")
        activity = activity.groupby(['patient_id','day_date']).filter(lambda x:len(x)>2)
        
    elif signal==1:
        print("Timestep: per week")
        activity['week'] = activity['day_date'].dt.to_period('W').apply(lambda r: r.start_time)
        activity.drop(['day_date','start_date'],axis=1, inplace=True)
        activity.columns=['patient_id','location_name','start_date']
        activity = activity.groupby(['patient_id','start_date']).filter(lambda x:len(x)>2)
        
    elif signal==2:
        print("Timestep: per hour")
        activity.start_date = activity.start_date.values.astype("datetime64[h]")
    
    elif signal==3:
        print("Accurate time")
        activity.start_date = activity.start_date.values.astype("datetime64[ns]")
        
    else:
        raise ValueError("Error: please input correct number! per day: 0, per week: 1")
    
    return activity

In [ ]:
# define network
class NEEP(nn.Module):
    def __init__(self, opt):
        super(NEEP, self).__init__()
        self.encoder = nn.Embedding(opt.n_token, opt.n_hidden)
        self.h = nn.Sequential()
        for i in range(opt.n_layer):
            self.h.add_module(
                'fc%d' % (i+1), nn.Linear(2*opt.n_hidden, 2*opt.n_hidden))
            self.h.add_module(
                'relu%d' % (i+1), nn.ReLU())
        self.h.add_module(
            'out', nn.Linear(2*opt.n_hidden, 1))

    def forward(self, s1, s2):
        s1 = self.encoder(s1)
        s2 = self.encoder(s2)
        x = torch.cat([s1, s2], dim=-1)
        _x = torch.cat([s2, s1], dim=-1)
        return self.h(x) - self.h(_x)


# define training and validation function
def train(opt, model, optim, trajs, sampler):
    model.train()
    batch, next_batch = next(sampler)

    s_prev = trajs[batch].to(opt.device)
    s_next = trajs[next_batch].to(opt.device)
    ent_production = model(s_prev, s_next)
    optim.zero_grad()
    
    # The objective function J. Equation (2)
    loss = (-ent_production + torch.exp(-ent_production)-1).mean()
    loss.backward()
    optim.step()
    return loss.item()


def validate(opt, model, trajs, sampler):
    model.eval()

    ret = []
    loss = 0
    with torch.no_grad():
        for batch, next_batch in sampler:
            s_prev = trajs[batch].to(opt.device)
            s_next = trajs[next_batch].to(opt.device)
            
            ent_production = model(s_prev, s_next)
            entropy = ent_production.cpu().squeeze().numpy()
            ret.append(entropy)
            loss += (- ent_production + torch.exp(-ent_production)-1).sum().cpu().item()
    loss = loss / sampler.size
    ret = np.concatenate(ret)
    ret = ret.reshape(trajs.shape[0], trajs.shape[1]-1)
    return ret, loss

In [ ]:
# 16 weeks for training, training timestep = 16 weeks, testing timestep = one week
def get_entropy_production(my_data):
    
    print(my_data.patient_id.iloc[1])
    # define training and testing data
    train_data = my_data[my_data['week_rank']<=16]
    trajs = np.array([(my_data[my_data['week_rank']<=16]).location_name]) # 16 weeks for training, the rest for testing

    # define parameters
    opt = Namespace()
    opt.device = 'cpu' 
    opt.n_token = 5
    opt.n_hidden = 32
    opt.n_layer = 1
    opt.batch_size = len(trajs[0])
    # opt.batch_size = 5
    opt.lr = 0.05
    # opt.lr = 0.01
    opt.wd = 5e-5

    opt.record_freq = 500
    opt.seed = 398

    torch.manual_seed(opt.seed)

    opt.M = len(trajs)             # number of trajectories
    opt.L = len(trajs[0])       # lenth of a trjectory 

    trajs_t = torch.from_numpy(trajs).long().view(1,-1,1)

    # training
    model = NEEP(opt)
    model = model.to(opt.device)
    optim = torch.optim.Adam(model.parameters(), opt.lr, weight_decay=opt.wd)

    train_sampler = CartesianSampler(opt.M, opt.L, opt.batch_size, device=opt.device)

    opt.n_iter = 100 # number of train ing iteration

    losses = []
    for i in tqdm(range(1, opt.n_iter + 1)):
        loss = train(opt, model, optim, trajs_t, train_sampler)
        if i%5==0:
            losses.append(loss)
    
    # testing
    prediction = pd.DataFrame(columns=['week', 'entropy_production'])
    my_week_date = []
    my_prediction = []
    test_data = my_data # take all of the data as the testing data
    for m in range(len(test_data.week_rank.unique())):
        my_week = test_data[test_data['week_rank']==m+1] # set the timestep of testing data to one week
        week_date = (np.array((test_data[test_data['week_rank']==m+1]).week))[0]
        week_result = []
        for mm in range(len(my_week.day_rank.unique())):
            test_trajs = np.array([(my_week[my_week['day_rank']==mm+1]).location_name])

            opt.M_t = len(test_trajs)  
            opt.L_t = len(test_trajs[0])
            opt.test_batch_size = len(test_trajs[0])

            test_trajs_t = torch.from_numpy(test_trajs).long().view(1,-1,1)
            test_sampler = CartesianSampler(opt.M_t, opt.L_t, opt.test_batch_size, device=opt.device, train=False)

            if (len(test_trajs[0])>10):
                pred, _ = validate(opt, model, test_trajs_t, test_sampler)
                pred = pred.flatten()
                cum_pred = np.cumsum(pred)

                slope, _, _, _, _ = stats.linregress(range(len(cum_pred)), cum_pred)
                week_result.append(slope)
        

        my_week_date.append(week_date)
        my_prediction.append(np.array(week_result).mean())

    prediction['entropy_production'] = my_prediction
    prediction['week'] = my_week_date
    prediction = prediction.reset_index(drop=True)
    
    return prediction

In [ ]:
activity_legacy = pre_process_legacy(activity_legacy)
activity_raw = pre_process_raw(activity_raw)
activity = pd.concat([activity_raw, activity_legacy], axis=0)

activity = select_daytime_night(activity, time)
activity = select_time_step(activity,timestep)
activity = activity.sort_values(['patient_id','day_date'])
activity = activity.reset_index(drop=True)

# revise location names
activity = activity.copy()
mapping = {
'bathroom':'0',
'bedroom':'1',
'hallway':'2',
'lounge':'3',
'kitchen':'4',
}
activity.location_name = activity.location_name.map(mapping)
activity.location_name = activity.location_name.values.astype('int')
activity['week'] = activity['day_date'].dt.to_period('W').apply(lambda r: r.start_time)

week_num = activity.groupby([activity['patient_id']]).apply(lambda x: len(x.week.unique()))
week_num.to_csv('middle.csv')
week_num = pd.read_csv('middle.csv')
week_num.columns = ['patient_id','week_num']

activity = pd.merge(activity,week_num,on='patient_id')

activity = activity[activity['week_num']>16] # at least 16 weeks for training
activity.drop(['week_num'],axis=1, inplace=True)
activity = activity.sort_values(['patient_id','day_date'])
activity = activity.reset_index(drop=True)
activity['week_rank'] = activity.groupby('patient_id')['week'].rank(method='dense')
activity.start_date = activity.start_date.values.astype('datetime64[D]')
activity.head()

In [ ]:
activity['day_rank'] = activity.groupby([activity['patient_id'], activity['week']])['day_date'].rank(method='dense')
activity.head()

In [ ]:
print(len(activity.patient_id.unique()))
etp = activity.groupby([activity['patient_id']]).apply(get_entropy_production)
etp.to_csv('middle.csv')
etp = pd.DataFrame(pd.read_csv('middle.csv'), columns = ['patient_id','week','entropy_production'])
etp = etp.groupby(['patient_id']).filter(lambda x:len(x)>8)
etp.head()

In [ ]:
etp.to_csv('c_activity_daytime_per_week_entropy_production.csv')